In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
file_train = Path(r"C:\devel\Loan_default_prediction\data\raw\application_train.csv").resolve()
file_test = Path(r"C:\devel\Loan_default_prediction\data\raw\application_test.csv").resolve()
df_train = pd.read_csv(file_train)
df_test = pd.read_csv(file_test)

In [3]:
def null_percentage(data):
    null_values_total = data.isnull().sum().sort_values(ascending = False)
    total_null_percent = (data.isnull().sum()/data.isnull().count() * 100).sort_values(ascending = False)
    return pd.concat([null_values_total, total_null_percent], axis = 1, keys = ['Total', 'Percent'])

In [4]:
print("The shape of the training dataset is: {}".format(df_train.shape))
print("The shape of the testing dataset is: {}".format(df_test.shape))

The shape of the training dataset is: (307511, 122)
The shape of the testing dataset is: (48744, 121)


In [5]:
# Dropping columns that contain more than 50 percent null values per feature
for column in df_train.columns:
    if null_percentage(df_train[column].to_frame())['Percent'][0] > 50.00:
        df_train.drop([column], axis = 1, inplace = True)
        df_test.drop([column], axis = 1, inplace = True)

In [6]:
def mean_imputation(data):
    """
    This function would take the NULL values present in data and replace them with average values of the data respectively.
    """
    imputed_data = data.fillna(data.mean()).copy()
    return imputed_data

def median_imputation(data):
    """
    This function would take the NULL values present in data and replace them with median values in our data respectively.
    """
    imputed_data = data.fillna(data.median()).copy()
    return imputed_data

def mode_imputation(data):
    """
    This function would take the NULL values present in data and replace them with mode values of the given data respectively.
    """
    imputed_data = data.fillna(data.mode()).copy()
    return imputed_data

In [7]:
mean_imputation(df_train['FLOORSMAX_AVG'])
mean_imputation(df_train['FLOORSMAX_MEDI'])
mean_imputation(df_train['FLOORSMAX_MODE'])
mean_imputation(df_train['YEARS_BEGINEXPLUATATION_AVG'])
mean_imputation(df_train['YEARS_BEGINEXPLUATATION_MEDI'])
mean_imputation(df_train['YEARS_BEGINEXPLUATATION_MODE'])

0         0.972200
1         0.985100
2         0.977065
3         0.977065
4         0.977065
            ...   
307506    0.978200
307507    0.972700
307508    0.981600
307509    0.977200
307510    0.988100
Name: YEARS_BEGINEXPLUATATION_MODE, Length: 307511, dtype: float64

In [8]:
null_percent_values = null_percentage(df_train)['Percent']
features_with_low_null_values = []
for i in range(len(df_train.columns)):
    if null_percent_values.iloc[i] < 10.0:
        features_with_low_null_values.append(null_percent_values.index[i]) 

In [9]:
df_train_converted = df_train[features_with_low_null_values]

In [10]:
## This is done to get the categorical features. We can then perform mode imputation to get the values respectively.
## The suggestion was implemented from the website https://stackoverflow.com/questions/29803093/check-which-columns-in-dataframe-are-categorical
total_columns =df_train_converted.columns 
numeric_data = df_train_converted._get_numeric_data()
categorical_columns = list(set(total_columns) - set(numeric_data.columns))

In [12]:
categorical_data = df_train_converted[categorical_columns]

In [13]:
mode_imputation(categorical_data)

,NAME_EDUCATION_TYPE,ORGANIZATION_TYPE,WEEKDAY_APPR_PROCESS_START,NAME_HOUSING_TYPE,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,NAME_INCOME_TYPE,CODE_GENDER,FLAG_OWN_REALTY,NAME_FAMILY_STATUS,NAME_TYPE_SUITE
0,Secondary / secondary special,Business Entity Type 3,WEDNESDAY,House / apartment,Cash loans,N,Working,M,Y,Single / not married,Unaccompanied
1,Higher education,School,MONDAY,House / apartment,Cash loans,N,State servant,F,N,Married,Family
2,Secondary / secondary special,Government,MONDAY,House / apartment,Revolving loans,Y,Working,M,Y,Single / not married,Unaccompanied
3,Secondary / secondary special,Business Entity Type 3,WEDNESDAY,House / apartment,Cash loans,N,Working,F,Y,Civil marriage,Unaccompanied
4,Secondary / secondary special,Religion,THURSDAY,House / apartment,Cash loans,N,Working,M,Y,Single / not married,Unaccompanied
...,...,...,...,...,...,...,...,...,...,...,...
307506,Secondary / secondary special,Services,THURSDAY,With parents,Cash loans,N,Working,M,N,Separated,Unaccompanied
307507,Secondary / secondary special,XNA,MONDAY,House / apartment,Cash loans,N,Pensioner,F,Y,Widow,Unaccompanied
307508,Higher education,School,THURSDAY,House / apartment,Cash loans,N,Working,F,Y,Separated,Unaccompanied
307509,Secondary / secondary special,Business Entity Type 1,WEDNESDAY,House / apartment,Cash loans,N,Commercial associate,F,Y,Married,Unaccompanied


In [ ]:
numeric_data.head()

In [ ]:
categorical_data.head()

In [ ]:
has_missing = numeric_data.isnull().any()
missing_data_columns = numeric_data.columns[has_missing]

In [ ]:
missing_data_columns

In [ ]:
# Showing data with features that have missing values
numeric_data[missing_data_columns].head()

In [ ]:
# Imputing the missing values with the mean of the data
mean_imputation(numeric_data[missing_data_columns])

In [ ]:
numeric_data.fillna(numeric_data.mean(), inplace = True)

In [ ]:
has_missing = categorical_data.isnull().any()
missing_data_columns = categorical_data.columns[has_missing]

In [ ]:
print("The shape of numeric data: {}".format(numeric_data.shape))
print("The shape of categorical data: {}".format(categorical_data.shape))

In [ ]:
# concatenating the categorical and numerical column
df_final = pd.concat([numeric_data, categorical_data], axis = 1)

In [ ]:
missing_data_columns